In [13]:
import os 
from langchain_google_genai import ChatGoogleGenerativeAI
from jinja2 import Template
from dotenv import load_dotenv
from langchain.output_parsers import StructuredOutputParser
load_dotenv()

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')


# Creating Reusable Prompt Templates
we will create a PromptTemplate class that uses Jinja2 for templating.
* Jinja2 provides more power,flexibility in template rendering than simple string formatting. 
* Jinja2 supports loops,contitionals,filters,readability and ideal for prompts, which lack in python string formatting.

In [3]:
class PromptTemplate:
     ''' A class to represent a template for generating prompts with variables
    Attributes:
        template (str): The template string with variables
        input_variables (list): A list of the variable names in the template
    '''
     def __init__(self,template,input_variables):
          self.template= Template(template)
          self.input_variables=input_variables

     def format(self, **kwargs):
          return self.template.render(**kwargs)

# Simple template with one variable
simple_template=PromptTemplate(
     template="Provide a brief explanation of {{topic}}.",
     input_variables=['topic']
) 

# More complex template with multiple variables
complex_template=PromptTemplate(
     template="Exmplain the concept of {{concept}} in the field of {{field}} to a {{audiance}} audiance, coniesly.",
     input_variables=['concept','field','audiance']
)

# Using simple template
print('Simple Template Result:')
prompt=simple_template.format(topic='Photosynthesis')
print(llm.invoke(prompt))

print('*'*500)

# Using complex template
print('Complex Template Result:')
prompt=complex_template.format(
     concept='Neural Networks',
     field='Artificial Intelligence',
     audience='Begineer'
)
print(llm.invoke(prompt))


Simple Template Result:
content='Photosynthesis is the process used by plants, algae, and some bacteria to convert light energy into chemical energy in the form of sugars.  This happens in two main stages: the light-dependent reactions, which capture light energy and convert it to chemical energy (ATP and NADPH), and the light-independent reactions (Calvin cycle), which use that energy to convert carbon dioxide into glucose.  In short, it\'s how plants "make their food" using sunlight, water, and carbon dioxide.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-8aa5d86a-4cb9-40f4-b938-ade6ff0a8994-0' usage_metadata={'input_tokens': 8, 'output_tokens': 102, 'total_tokens': 110, 'input_token_details': {'cache_read': 0}}
********************************************************************************************************************************************************************

# 2. Using Variables for Dynamic Content.

In [5]:
# Template with conditional content
conditional_template= PromptTemplate(
    template="My name is {{ name }} and I am {{ age }} years old. "
              "{% if profession %}I work as a {{ profession }}.{% else %}I am currently not employed.{% endif %} "
              "Can you give me career advice based on this information? answer concisely.",
    input_variables=['name','age','profession']
)

# Using conditional template
print('Conditional Template Result (with profession)')
prompt=conditional_template.format(
    name='Akira',
    age='28',
    profession='Software developer'
)
print(llm.invoke(prompt))

print("*"*500)

print('\n Conditional Template Result (without profession)')
prompt=conditional_template.format(
    name='Sam',
    age='22',
    profession=''
)
print(llm.invoke(prompt))



Conditional Template Result (with profession)
content='Akira, focus on specialization (e.g., AI, cloud, cybersecurity), continuous learning (courses, certifications), networking (conferences, online communities), and building a strong portfolio showcasing your skills. Consider leadership roles or freelancing as potential paths.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-c0da4c0e-e424-4285-9194-762e22132d7c-0' usage_metadata={'input_tokens': 35, 'output_tokens': 51, 'total_tokens': 86, 'input_token_details': {'cache_read': 0}}
*****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [12]:
# Template for list processing
list_template=PromptTemplate(
    template="Categorize these items into groups: {{ items }}. Provide the categories and the items in each category.",
    input_variables=['items']
)
# Using list template
print('List Template Result:')
prompt=list_template.format(
    items='apple, banana, carrot, hammer, screwdriver, pliers, novel, textbook, magazine'

)
print(llm.invoke(prompt))

List Template Result:
content="Here's a categorization of the items:\n\n**Category 1: Fruit**\n* Apple\n* Banana\n\n**Category 2: Vegetables**\n* Carrot\n\n**Category 3: Hand Tools**\n* Hammer\n* Screwdriver\n* Pliers\n\n**Category 4: Printed Materials**\n* Novel\n* Textbook\n* Magazine" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-1740eede-d883-4a6d-a4f7-b420d2bc91f9-0' usage_metadata={'input_tokens': 35, 'output_tokens': 73, 'total_tokens': 108, 'input_token_details': {'cache_read': 0}}


# Advanced Template Techniques

In [14]:
list_format_template=PromptTemplate(
    template="Analyze the following list of items:\n"
              "{% for item in items.split(',') %}"
              "- {{ item.strip() }}\n"
              "{% endfor %}"
              "\nProvide a summary of the list and suggest any patterns or groupings.",
    input_variables=["items"]
)

# Using the formatted list template
print("Formatted list Template Result:")
prompt=list_format_template.format(
    items='Python, JavaScript, HTML, CSS, React, Django, Flask, Node.js'
)
print(llm.invoke(prompt))

Formatted list Template Result:
content="This list contains a mix of programming languages, scripting languages, and front-end/back-end web development frameworks.  We can group them as follows:\n\n**1. Programming/Scripting Languages:**\n\n* **Python:** A general-purpose, high-level programming language used extensively in web development (backend), data science, machine learning, and more.\n* **JavaScript:** A primarily client-side scripting language for web browsers, increasingly used on the server-side (Node.js).\n* **HTML (HyperText Markup Language):**  Not a programming language, but a markup language used to structure web pages.  It's foundational to web development but doesn't contain logic.\n* **CSS (Cascading Style Sheets):**  Not a programming language either, but a stylesheet language used to style HTML elements, controlling the visual presentation of web pages.\n\n\n**2. Web Development Frameworks/Libraries:**\n\n* **React:** A JavaScript library for building user interfac

In [15]:
# Template with dynamic instructions
dynamic_instruction_template = PromptTemplate(
    template="Task: {{ task }}\n"
              "Context: {{ context }}\n"
              "Constraints: {{ constraints }}\n\n"
              "Please provide a solution that addresses the task, considers the context, and adheres to the constraints.",
    input_variables=["task", "context", "constraints"]
)

# Using the dynamic instruction template
print("Dynamic instruction template result")
prompt=dynamic_instruction_template.format(
    task='Design a logo for a tech startup',
    context='The startup focuses on AI-driven healthcare solutions',
    constraints='Must use blue and green colors,and should be simple enough to be recognizaalbe when small'
)
print(llm.invoke(prompt))

Dynamic instruction template result
content='## Logo Concept:  "Synaptic Bloom"\n\n**Concept:**  The logo combines the imagery of a synapse (representing AI and neural networks) with a stylized flower (representing growth, health, and life). This visually connects the AI technology with its application in healthcare.\n\n**Visual Elements:**\n\n* **Shape:** A stylized abstract flower with five petals, each petal subtly curving to resemble a synapse\'s branching structure.  The petals are not perfectly symmetrical for a more organic and less clinical feel.\n* **Color:** A gradient blend of teal (a mix of blue and green) forms the majority of the flower.  The center of the flower can be a slightly darker teal or a deep, muted green to provide contrast and focus.\n* **Simplicity:** The overall shape is simple and easily recognizable even at small sizes.  There are no unnecessary details or complex lines.\n\n**Color Palette:**\n\n* **Main Color:** #008080 (Teal) – a calming and trustworthy 